In [1]:
# Import required libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Set the path to your training and validation data
train_path = '/kaggle/input/all-dataset/All dataset/synthetic+real/train'
valid_path = '/kaggle/input/all-dataset/All dataset/synthetic+real/test'

In [3]:
# Define the batch size and input size
batch_size = 32
input_size = (224, 224)

In [4]:
# Create an instance of the ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [5]:
# Load the training data using the generator
train_data = train_datagen.flow_from_directory(
        train_path,
        target_size=input_size,
        batch_size=batch_size,
        class_mode='binary')

Found 940 images belonging to 2 classes.


In [6]:
# Create an instance of the ImageDataGenerator for validation data
valid_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
# Load the validation data using the generator
valid_data = valid_datagen.flow_from_directory(
        valid_path,
        target_size=input_size,
        batch_size=batch_size,
        class_mode='binary')

Found 200 images belonging to 2 classes.


In [8]:
# Create a new sequential model
model = Sequential()

In [9]:
# Add a convolutional layer with 32 filters, a 3x3 kernel size, and ReLU activation
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(input_size[0], input_size[1], 3)))

# Add a max pooling layer with 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another convolutional layer with 64 filters, a 3x3 kernel size, and ReLU activation
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add a max pooling layer with 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output from the convolutional layers
model.add(Flatten())

# Add a fully connected layer with 128 neurons and ReLU activation
model.add(Dense(128, activation='relu'))

# Add a dropout layer with a rate of 0.5 to reduce overfitting
model.add(Dropout(0.5))

# Add the output layer with a single neuron and sigmoid activation
model.add(Dense(1, activation='sigmoid'))

# Compile the model with an optimizer, loss function, and metrics to monitor
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 186624)            0         
                                                                 
 dense (Dense)               (None, 128)               2

In [11]:
# Train the model on the training data
history = model.fit(
        train_data,
        steps_per_epoch=train_data.samples // batch_size,
        epochs=100,
        validation_data=valid_data,
        validation_steps=valid_data.samples // batch_size)

Epoch 1/100
29/29 [==============================] - 176s 6s/step - loss: 2.4069 - accuracy: 0.4923 - val_loss: 0.6932 - val_accuracy: 0.4948
Epoch 2/100
29/29 [==============================] - 125s 4s/step - loss: 0.6931 - accuracy: 0.5066 - val_loss: 0.6930 - val_accuracy: 0.5104
Epoch 3/100
29/29 [==============================] - 129s 4s/step - loss: 0.6932 - accuracy: 0.4989 - val_loss: 0.6927 - val_accuracy: 0.5677
Epoch 4/100
29/29 [==============================] - 123s 4s/step - loss: 0.6979 - accuracy: 0.5452 - val_loss: 0.6864 - val_accuracy: 0.6146
Epoch 5/100
29/29 [==============================] - 125s 4s/step - loss: 0.6896 - accuracy: 0.5496 - val_loss: 0.6860 - val_accuracy: 0.5781
Epoch 6/100
29/29 [==============================] - 124s 4s/step - loss: 0.6777 - accuracy: 0.5925 - val_loss: 0.6839 - val_accuracy: 0.5260
Epoch 7/100
29/29 [==============================] - 129s 4s/step - loss: 0.6660 - accuracy: 0.6079 - val_loss: 0.6749 - val_accuracy: 0.5938
Epoch 

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on the validation data
loss, acc = model.evaluate(valid_data, verbose=0)
print('Validation accuracy: {:.2f}%'.format(acc*100))

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Accuracy Graph')
plt.legend()
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Loss Graph')
plt.legend()
plt.show()